# Model training for detecting incoming vehicles.

This model will be trained to recognize incoming vehicles by detecting frontal features of a car such as the bumper, headlight, windscreen and rear mirrors.

# Mounting Google Drive

Importing the drive module from the google.colab library to use the module's functions to interact with my Google Drive within the Colab notebook.

Mounting Google Drive to the specified directory (/content/drive) in the Colab environment to access the files and folders stored in my Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Ultralytics

Importing Ultralytics library, which is the official library for the YOLO (You Only Look Once) object detection models, in our case, we will be using YOLOv11.
It'll includes tools for our model training, validation, and inference.

In [2]:
# Installs the Ultralytics library

# > /dev/null ensures that the installation messages, such as progress bars and success notifications, are suppressed.
%pip install ultralytics > /dev/null

In [3]:
# Imports the Ultralytics library
import ultralytics

# Checks the current environment for compatibility and readiness to work with the Ultralytics library
ultralytics.checks()

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 32.6/235.7 GB disk)


# Hyperparameter Tuning (Grid Search)

Experimenting different batch sizes (8, 16, 32, 64, 128) with 2 different learning rates (0.01, 0.005)

In [4]:
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [0.01, 0.005]

In [5]:
from ultralytics import YOLO

In [6]:
# A UTF-8 locale is required.
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [10]:
for batch in batch_sizes:
    for lr in learning_rates:
        # YOLOv11 pre-trained model is reinitialized from the pre-trained weights (yolo11n.pt) at the start of each batch size experiment to test each batch size, learning rate combination independently
        model = YOLO("yolo11n.pt")
        print("\n#################################################")
        print(f"Training on batch size {batch} and learning rate {lr}")
        print("#################################################\n")

        # Define results directory in Google Drive
        results_dir = "/content/drive/My Drive/front-vehicle-detect-results"
        run_name = f"bs{batch}_lr{lr}"
        val_name = f"val_bs{batch}_lr{lr}"

        train_result = model.train(
            data="/content/drive/My Drive/front-vehicle-detect-cv-1/data.yaml",
            epochs=100,
            imgsz=640,
            device="gpu",
            batch=batch,            # Adjust batch size
            lr0=lr,                 # Adjust learning rate
            project=results_dir,    # Save directly to Drive folder
            name=run_name,          # Each combination gets its own folder
        )
        model.val(
            project=results_dir,
            name=val_name
        )


#################################################
Training on batch size 8 and learning rate 0.01
#################################################

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:gpu (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/drive/My Drive/front-vehicle-detect-cv-1/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=/content/drive/My Drive/front-vehicle-detect-results, name=bs8_lr0.01, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stri

train: Scanning /content/drive/My Drive/front-vehicle-detect-cv-1/train/labels.cache... 521 images, 166 backgrounds, 0 corrupt: 100%|██████████| 527/527 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/My Drive/front-vehicle-detect-cv-1/valid/labels.cache... 102 images, 38 backgrounds, 0 corrupt: 100%|██████████| 102/102 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/front-vehicle-detect-results/bs8_lr0.01/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/front-vehicle-detect-results/bs8_lr0.01
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


100 epochs completed in 0.131 hours.
Optimizer stripped from /content/drive/My Drive/front-vehicle-detect-results/bs128_lr0.005/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/My Drive/front-vehicle-detect-results/bs128_lr0.005/weights/best.pt, 5.5MB

Validating /content/drive/My Drive/front-vehicle-detect-results/bs128_lr0.005/weights/best.pt...
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:gpu (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.53it/s]


                   all        102        613      0.834      0.653       0.75      0.401
Speed: 0.2ms preprocess, 0.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /content/drive/My Drive/front-vehicle-detect-results/bs128_lr0.005
Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:gpu (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/drive/My Drive/front-vehicle-detect-cv-1/valid/labels.cache... 102 images, 38 backgrounds, 0 corrupt: 100%|██████████| 102/102 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.69s/it]


                   all        102        613      0.816      0.665       0.75      0.401
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/My Drive/front-vehicle-detect-results/val_bs128_lr0.005
